# Webshop - DDL - Anlegen der Tabellen und Beziehungen
### Hinweise
- In den Verbindungsparametern müssen Sie unter hostname die private IP-Adresse des MySQL-Servers einfügen (siehe Codelab).
- Auch das Passwort des root-Nutzers müssen Sie anpassen.
- Wenn Sie die gesamte webshop-Datenbankstruktur erzeugen möchten müssen Sie alle Zellen ausführen (Menü Run -> Run all cells).

In [ ]:
# verwendete Pakete
import mysql.connector

### Herstellen der Datenbankverbindung

#### Notwendige Verbindungsparameter

In [ ]:
hostname = ''  # die private IP-Adresse der Cloud SQL-Instanz
username = 'root'  # wir haben bisher nur root als Nutzer
password = ''  # das Kennwort, das Sie beim Anlegen der Instanz vergeben haben
database = 'webshop'  # dies ist unsere Webshop-Datenbank

### Testen der Verbindung

In [ ]:
# dieser code sollte die bisherigen Datenbanken in Ihrem RDBMS anzeigen
conn = mysql.connector.connect( host=hostname, user=username, passwd=password, autocommit=True)
cur = conn.cursor()

cur.execute(f"show databases")

print(f"Databases:")
for db in cur.fetchall():
    print(f"- {db[0]}")
    
conn.close()

## Anlegen der Datenbank und Definition des Datenbankschemas

In [ ]:
# read database and table structure from SQL file
ddl_filename = "DSCB140 - Fallstudie Webshop - DDL.sql"
with open (ddl_filename, "r") as ddl_file:
    ddl_commands=ddl_file.readlines()
    ddl_commands = "".join(ddl_commands)

In [ ]:
conn = mysql.connector.connect( host=hostname, user=username, passwd=password, autocommit=True)  # create connection

cur = conn.cursor()
cur.execute(ddl_commands, multi=True)  # execute all SQL-statements from file 
    
conn.close()


## Anzeigen der Tabellen in der Datenbank "Webshop"

In [ ]:
conn = mysql.connector.connect( host=hostname, user=username, passwd=password, db=database, autocommit=True)
cur = conn.cursor()

cur.execute(f"show tables")

print(f"Tables in database {database}:")
for tbl in cur.fetchall():
    print(f"- {tbl[0]}")
    
conn.close()
